# Using databases with python- week 3

From [coursera](https://www.coursera.org/learn/python-databases).

July 2022. 

In [1]:
%%javascript
$.getScript('https://kmahelona.github.io/ipython_notebook_goodies/ipython_notebook_toc.js')

<IPython.core.display.Javascript object>

<div id="toc"></div>

# Data models

## Designing a data model

Now we talk about linking tables together. Each table will have a number of columns, some of the columns in the table are special columns that represent relationships to the other tables. 

This is where the power and complexity comes in. Teams should take time to carefully design the database. Draw a picture of the data objects for our application, and then figureo ut how to represent the objects and their relationships. 

**Basic rule**: Don't put the same data in twice- use a relationship instead.

For each piece of information:  
* Does this represent an object? Or an attribute of another object?  
* Once we define objects, we need to define the relationships between objects  

Create a list of each piece of information (columns):  
* Columns: Track title / album / artist / genre / rating / len / count  
* **FIRST TABLE**: Tracks  
    * Start with the thing that is the most essential to this application. In this example, we are managing music tracks. Each row will have one unique track. So the first table will be the track table.  
    * Which columns are attributes of the track? length, rating, count.  
    * Columns in first table: track title / length / rating / count  
* **SECOND TABLE**: Albums    
    * What's the next thing to draw? Go to the albums. Tracks belong to albums, then albums belong to artists.  
* **OTHER COLUMNS**
    * Where does genre belong? Is it connected to an album, artist, or track? In most music software, changing the genre only affects that one track. So go back and add genre to the original track table. 
* **ID Key**  
    * Add a primary key to each table. The primary key in each table is unique to that table only.  
        * There is one key for every row.  
        * When you set up the table, you can set it as a primary key, and tell it to auto increment so it automatically numbers up starting at 1.  
    * If you want to link to another table, you can add a foreign key, which points to the primary key of another table. Name this key in a descriptive way.   
    * A logical key is a way of setting up a relationship that you might use  
    

<img src="img\week3_01.jpg?modified=2" />

## Setting up a data model in tables

Start setting up a local db like this. Start by creating the table `tracks`. 

First, create your primary key `id`. Tell it to autoincrement. 

<img src="img\week3_02.jpg?modified=1" />

I can create the other columns, although I can't create the columns for foreign keys because I have to be able to link them to their associated primary keys. 

I made a mistake here. You want to work from the outside of the tree toward the inside. I should have started with the artist table, then album, genre, and tracks last. 

I'll redo it in this order. Now, when I get to the album table, I can set up `artist_id` as a foreign key, linked to `id` in the `artist` table. 

<img src="img\week3_03.jpg?modified=1" />

We can also create tables using the SQL command. For example, creating the `album` table above would take this command:  

`CREATE TABLE album (
    id        INTEGER PRIMARY KEY ASC AUTOINCREMENT,
    title     TEXT,
    artist_id INTEGER REFERENCES artist (id) 
);`

I'm going to remote the `tracks` table and create it that way. 

`CREATE TABLE tracks(
    id     INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT UNIQUE,  
    title  TEXT,  
    album_id INTEGER REFERENCES artist (id),  
    genre_id INTEGER REFERENCES genre (id),  
    len INTEGER,  
    rating INTEGER,  
    count INTEGER  
)`

If I need to remove a table to redo the `CREATE` command from scratch, I can use:  

`DROP TABLE tracks`

## Insert data into the tables

Case-sensitive?  
* SQL commands- no  
* Names of tables and columns- maybe  
* Data itself- yes  

Insert an entry into the `artist` table. The `id` field will be auto-generated, so we do not need to specify a value for that. 

`INSERT INTO artist (name) VALUES ('Taylor Swift')`

When we look at the artist table and refresh, we see one entry with `id = 1`.  Each subsequent addition will have the next integer value as `id`. 

I'm going to add a few more artists in a row. Be sure to include a semi-colon at the end of each command. 

<img src="img\week3_04.jpg?modified=1" />

Now let's add some data to the `genre` table. This is just a list of genres. So far we have artists who have written a number of genres, but let's just add pop, indie, and country. I'm going to try adding all three at once. You must have parenthesis around each individual item.

`INSERT INTO genre (name) VALUES ('Pop'),('Indie'),('Country')`

We need to pay attention to the `id` values here because we are going to link them to the other tables.   
<img src="img\week3_05.jpg?modified=1" />

To add data to the album table, we need to consider all three columns that we have set up:  
* `id` - will be auto-populated  
* `title` - we will specify  
* `artist_id` - we will specify, needs to match `id` in the `artist` table  

So let's add the Sour album. 

`INSERT INTO album (title, artist_id) VALUES ('Sour',2)`

For foreign keys, we have to put in the numbers explicitly. It may seem easier to put the names in, but remember that the computer is going to keep track of everything, so we won't usually put these in by hand. 

<img src="img\week3_06.jpg?modified=1" />

Finally, let's popular the `tracks` table. I'll add my favorite songs from each of these albums. 

First, I tried the following commands, but they failed. SQL was complaining about a foreign key, though I didn't know which one. It turns out that the `album key` with `id=5` was failing, because I had accidentally set up this foreign key to be connected to the `artist` `id` value. Big mistake! I made the change and then hit commit, which launched a lengthy SQL command. 

Now, rerunning the SQL command, the entries are created.

<img src="img\week3_07.jpg?modified=3" />

We have replication of numbers in the `album_id` and `genre_id` columns, but that's ok. The important thing is that the words `Pop` and `Indie` only appear once in the database. Integers are must easier to store than strings over and over.

## Look at the final product

Just to visualize what we have created, let's print everything out. 

<img src="img\week3_08.jpg?modified=1" />

I made that diagram myself, but some database programs can also visualize all of the relationships for you. I'm going to try with the program [`DBeaver`](https://github.com/dbeaver/dbeaver/wiki)

It's not as pretty as my diagram, but it works out of the box. Open the database, select everything in the DatabaseNavigator panel, right click and select `Create New ER Diagram`. Give it a name and voila! 

<img src="img\week3_12.jpg?modified=1" />

# Reconstructing data with `JOIN`

Now we have all of our data in the tables. Let's come full circle and bring it all back together with the `JOIN` command. 

We need to reconstruct all of these relationships. Select data from multiple tables and connect tables to one another. 

## Combine album title and artist name

We have bits and pieces of information all over the place, but what if we want to display a UI like what you'd see in iTunes, that combines all of the info for each track? We need to combine them together. 



Start by creating a table of album title and artist name.  
* Album title will come from `album` table  
* Artist name will come from `artist` table  
* We'll connect the `album` table to the `artist` table using the `artist_id` foreign key  

We will need to pass the following info:  
* What we want to see: `SELECT album.title, artist.name`  
* The tables that hold the data: `FROM album JOIN artist`  
* How the tables are linked: `ON album.artist_id = artist.id`  

<img src="img\week3_09.jpg?modified=1" />

Right now this table just lives in our temporary results field, passed back to us from the SQL query. It is not stored anywhere. We're not building a new table in the database right now. We are just retrieving information from the database and putting it into a convenient form locally.

Notice that we don't have any of the id keys here. That's because we don't need them here anymore. We could have asked for them if we wanted them, but nahh. 

## Combine track title and genre name

Step through our query again:  
* What we want to see: `SELECT tracks.title, genre.name`  
* The tables that hold the data: `FROM tracks JOIN genre`  
* How the tables are linked: `ON tracks.genre_id = genre.id`  

<img src="img\week3_10.jpg?modified=1" />

Looks great. We are getting the hang of it. However, the naming here is confusing because we use the columns `name` and `title` in multiple tables. So let's rename them:  

<img src="img\week3_11.jpg?modified=1" />

## Try it without the `ON` clause

What's going on under the hood? Try running this without the `ON` clause- it will try all combinations. 

<img src="img\week3_13.jpg?modified=1" />

So the `JOIN` operation is actually generating all possible combinations, and then the `ON` clause is telling it to only keep the correct entries. 

## Combine all the tables together

We want one final table with the following columns:  
* track title  
* artist name  
* album title  
* genre name

Let's follow our steps for building the query: 
* What we want to see: `SELECT tracks.title, artist.name, album.title, genre.name`  
* The tables that hold the data: `FROM tracks JOIN artist JOIN album JOIN genre`  
* How the tables are linked: `ON tracks.genre_id = genre.id and tracks.album_id = album.id and album.artist_id = artist.id`  

Spacing in the SQL query does not matter, so I am going to organize the text so that I can visually parse all of the lines quickly and clearly. 

<img src="img\week3_14.jpg?modified=1" />

Now we are reconstructing the view that our users actually want to see, but we can also handle millions and billions of rows because we have constructed a data model that makes sense. 